# Module 07: Reproducible Research

**Estimated Time:** 40 minutes

## Learning Objectives

By the end of this module, you will be able to:

1. ✅ Understand what reproducibility means in data science research
2. ✅ Set up reproducible computational environments
3. ✅ Organize code for reproducibility and maintainability
4. ✅ Control randomness in experiments
5. ✅ Manage dependencies and versions effectively
6. ✅ Apply best practices for data versioning

## Why Reproducibility Matters

**Reproducibility** is the cornerstone of trustworthy research. If other researchers (or even you in 6 months!) cannot reproduce your results, the findings lose credibility.

### Real-World Impact:
- 🔬 **Science Crisis**: Studies estimate that 50-90% of published research may not be reproducible
- 💰 **Cost**: Irreproducible research costs billions in wasted resources
- 🏆 **Career**: Reproducible work builds reputation and enables collaboration
- ⚖️ **Legal**: In healthcare, finance, and criminal justice, reproducibility can be legally required

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import platform
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Create output directory
output_dir = "outputs/module07"
os.makedirs(output_dir, exist_ok=True)

print("✅ Module 07: Reproducible Research - Setup Complete!")
print(f"📁 Outputs will be saved to: {output_dir}")

---

## Part 1: Understanding Reproducibility

### Types of Reproducibility

There are three main types:

#### 1. **Computational Reproducibility** 🖥️
- Same data + Same code + Same environment = **Same results**
- This is the minimum standard for data science
- Focus: Environment, dependencies, random seeds

#### 2. **Empirical Reproducibility** 🔬
- Same methodology + New data collection = **Consistent findings**
- Tests if the effect is real, not just a fluke
- Focus: Methodology documentation, protocols

#### 3. **Statistical Reproducibility** 📊
- Same data + Different analysis = **Robust conclusions**
- Tests sensitivity to analytical choices
- Focus: Transparency about decisions, reporting all analyses

### The Reproducibility Spectrum

```
Not Reproducible ←―――――――――――――――――――――→ Fully Reproducible
       |
       |-- No documentation
       |-- Manual steps
       |-- Hard-coded paths
       |-- Version conflicts
       |                  |
       |                  |-- Basic documentation
       |                  |-- Some automation
       |                  |-- Relative paths
       |                  |                     |
       |                  |                     |-- Complete documentation
       |                  |                     |-- Fully automated
       |                  |                     |-- Version control
       |                  |                     |-- Containerized
       |                  |                     |-- Public data/code
```

In [ ]:
# Example: Demonstrating non-reproducible vs reproducible code

print("=" * 60)
print("NON-REPRODUCIBLE CODE EXAMPLE")
print("=" * 60)

# ❌ BAD: Non-reproducible
bad_example = """
# No random seed set
data = np.random.randn(100)  
# Hard-coded absolute path
df = pd.read_csv('C:/Users/John/data.csv')  
# No version info
from sklearn.model import MyModel  
"""
print(bad_example)

print("\n" + "=" * 60)
print("REPRODUCIBLE CODE EXAMPLE")
print("=" * 60)

# ✅ GOOD: Reproducible
good_example = """
# Set random seed for reproducibility
np.random.seed(42)  
data = np.random.randn(100)

# Use relative paths
data_dir = os.path.join('..', 'data')
df = pd.read_csv(os.path.join(data_dir, 'data.csv'))

# Document versions in requirements.txt
# scikit-learn==1.3.0
from sklearn.model import MyModel
"""
print(good_example)

In [ ]:
# Visualizing reproducibility challenges

challenges = {
    "Environment\nDifferences": 35,
    "Random\nSeeds": 25,
    "Data\nVersioning": 20,
    "Dependency\nConflicts": 15,
    "Documentation\nGaps": 5,
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
colors = sns.color_palette("Set2", len(challenges))
ax1.pie(
    challenges.values(), labels=challenges.keys(), autopct="%1.1f%%", startangle=90, colors=colors
)
ax1.set_title("Common Reproducibility Challenges\n(% of issues)", fontsize=14, fontweight="bold")

# Bar chart
bars = ax2.barh(list(challenges.keys()), list(challenges.values()), color=colors)
ax2.set_xlabel("Percentage (%)", fontsize=12)
ax2.set_title("Reproducibility Challenges by Type", fontsize=14, fontweight="bold")
ax2.grid(axis="x", alpha=0.3)

# Add value labels
for i, (bar, value) in enumerate(zip(bars, challenges.values())):
    ax2.text(value + 1, i, f"{value}%", va="center", fontsize=10)

plt.tight_layout()
plt.savefig(
    os.path.join(output_dir, "reproducibility_challenges.png"), dpi=300, bbox_inches="tight"
)
plt.show()

print(
    "\n💡 Key Insight: Environment differences and random seeds cause 60% of reproducibility issues!"
)

---

## Part 2: Reproducible Environments

### 📦 Virtual Environments

Virtual environments isolate project dependencies, preventing conflicts.

#### Why Use Virtual Environments?
- ✅ **Isolation**: Different projects can use different library versions
- ✅ **Portability**: Easy to share exact environment setup
- ✅ **Reproducibility**: Ensures consistent results across machines

#### Common Tools:

| Tool | Best For | Pros | Cons |
|------|----------|------|------|
| **venv** | Python-only projects | Built-in, simple | Python only |
| **conda** | Data science, multi-language | Handles non-Python deps | Slower, larger |
| **Docker** | Production, complex stacks | Complete isolation | Steeper learning curve |

### Creating Virtual Environments

```bash
# Using venv (Python 3.3+)
python -m venv myenv
source myenv/bin/activate  # On Windows: myenv\Scripts\activate
pip install -r requirements.txt

# Using conda
conda create -n myenv python=3.10
conda activate myenv
conda install --file requirements.txt

# Using Docker
docker build -t my-research-env .
docker run -v $(pwd):/workspace my-research-env
```

In [ ]:
# Document your current environment


def document_environment():
    """
    Capture comprehensive environment information.

    This is crucial for reproducibility!
    """
    env_info = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "python_version": sys.version,
        "platform": platform.platform(),
        "machine": platform.machine(),
        "processor": platform.processor(),
    }

    # Get package versions
    import importlib.metadata

    key_packages = ["numpy", "pandas", "matplotlib", "seaborn", "scikit-learn", "scipy"]

    package_versions = {}
    for pkg in key_packages:
        try:
            package_versions[pkg] = importlib.metadata.version(pkg)
        except importlib.metadata.PackageNotFoundError:
            package_versions[pkg] = "Not installed"

    return env_info, package_versions


# Document current environment
env_info, packages = document_environment()

print("=" * 60)
print("ENVIRONMENT DOCUMENTATION")
print("=" * 60)
print(f"\n📅 Timestamp: {env_info['timestamp']}")
print(f"🐍 Python: {env_info['python_version'].split()[0]}")
print(f"💻 Platform: {env_info['platform']}")
print(f"🖥️  Machine: {env_info['machine']}")

print("\n📦 Package Versions:")
print("-" * 60)
for pkg, version in packages.items():
    print(f"  {pkg:20s}: {version}")

# Save to file
env_df = pd.DataFrame(
    [{"Category": "System", "Item": k, "Value": v} for k, v in env_info.items()]
    + [{"Category": "Package", "Item": k, "Value": v} for k, v in packages.items()]
)

env_df.to_csv(os.path.join(output_dir, "environment_info.csv"), index=False)
print(f"\n✅ Environment info saved to {output_dir}/environment_info.csv")

In [ ]:
# Generate a requirements.txt template


def generate_requirements_template():
    """
    Generate a well-documented requirements.txt file.
    """
    template = """# Research Project Dependencies
# Generated: {timestamp}
# Python Version: {python_version}

# Core Scientific Computing
numpy>=1.24.0
pandas>=2.0.0
scipy>=1.10.0

# Visualization
matplotlib>=3.7.0
seaborn>=0.12.0

# Machine Learning
scikit-learn>=1.3.0

# Statistical Analysis
statsmodels>=0.14.0

# Jupyter
jupyter>=1.0.0
notebook>=7.0.0
ipywidgets>=8.0.0

# Utility
python-dateutil>=2.8.0
pytz>=2023.3

# Development (optional)
# pytest>=7.4.0
# black>=23.0.0
# flake8>=6.0.0
""".format(
        timestamp=datetime.now().strftime("%Y-%m-%d"), python_version=sys.version.split()[0]
    )

    return template


requirements = generate_requirements_template()
print("📄 REQUIREMENTS.TXT TEMPLATE")
print("=" * 60)
print(requirements)

# Save template
with open(os.path.join(output_dir, "requirements_template.txt"), "w") as f:
    f.write(requirements)

print(f"\n✅ Template saved to {output_dir}/requirements_template.txt")

---

## Part 3: Controlling Randomness

### 🎲 Random Seeds

Many data science operations involve randomness:
- Train/test splits
- Random initialization of models
- Data shuffling
- Random sampling

**Setting random seeds ensures these operations are reproducible.**

### Best Practices:
1. ✅ Set seeds at the **start** of your notebook/script
2. ✅ Set seeds for **all** libraries you use (NumPy, Python random, TensorFlow, PyTorch, etc.)
3. ✅ Document the seed value in your code and paper
4. ✅ Use the **same seed** across related experiments for fair comparison

In [ ]:
# Demonstrate the importance of random seeds

print("=" * 60)
print("DEMONSTRATION: Effect of Random Seeds")
print("=" * 60)

# WITHOUT seed (non-reproducible)
print("\n❌ WITHOUT SEED (Run this cell multiple times - results change!)")
data_no_seed = np.random.randn(5)
print(f"Random data: {data_no_seed}")

# WITH seed (reproducible)
print("\n✅ WITH SEED (Run this cell multiple times - results stay the same!)")
np.random.seed(42)
data_with_seed = np.random.randn(5)
print(f"Random data: {data_with_seed}")

In [ ]:
# Comprehensive seed setting function


def set_all_seeds(seed=42):
    """
    Set random seeds for all common libraries.

    Parameters:
    -----------
    seed : int
        The random seed value (default: 42)

    Note:
    -----
    Call this function at the START of your notebook/script!
    """
    # Python's built-in random
    import random

    random.seed(seed)

    # NumPy
    np.random.seed(seed)

    # TensorFlow (if installed)
    try:
        import tensorflow as tf

        tf.random.set_seed(seed)
        print(f"✅ TensorFlow seed set to {seed}")
    except ImportError:
        pass

    # PyTorch (if installed)
    try:
        import torch

        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        print(f"✅ PyTorch seed set to {seed}")
    except ImportError:
        pass

    print(f"✅ Python random seed set to {seed}")
    print(f"✅ NumPy seed set to {seed}")

    return seed


# Set all seeds
RANDOM_SEED = 42  # Document this clearly!
set_all_seeds(RANDOM_SEED)

print(f"\n🔒 All random operations are now reproducible with seed={RANDOM_SEED}")

In [ ]:
# Visualize reproducibility with seeds

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Create synthetic data
np.random.seed(42)
X = np.random.randn(100, 1)
y = 3 * X.squeeze() + 2 + np.random.randn(100) * 0.5

# Compare results with and without seed
results = []

for trial in range(5):
    # Without seed - different split each time
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = LinearRegression().fit(X_train, y_train)
    score_no_seed = model.score(X_test, y_test)

    # With seed - same split every time
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression().fit(X_train, y_train)
    score_with_seed = model.score(X_test, y_test)

    results.append(
        {"Trial": trial + 1, "Without Seed": score_no_seed, "With Seed": score_with_seed}
    )

results_df = pd.DataFrame(results)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(results_df))
width = 0.35

bars1 = ax.bar(
    x - width / 2,
    results_df["Without Seed"],
    width,
    label="Without Seed (Non-reproducible)",
    alpha=0.8,
    color="coral",
)
bars2 = ax.bar(
    x + width / 2,
    results_df["With Seed"],
    width,
    label="With Seed (Reproducible)",
    alpha=0.8,
    color="lightblue",
)

ax.set_xlabel("Trial", fontsize=12)
ax.set_ylabel("R² Score", fontsize=12)
ax.set_title(
    "Impact of Random Seeds on Model Performance\n(Same data, different train/test splits)",
    fontsize=14,
    fontweight="bold",
)
ax.set_xticks(x)
ax.set_xticklabels(results_df["Trial"])
ax.legend()
ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "seed_impact.png"), dpi=300, bbox_inches="tight")
plt.show()

print("\n📊 Results Summary:")
print(results_df.to_string(index=False))
print(f"\nWithout Seed - Std Dev: {results_df['Without Seed'].std():.6f}")
print(f"With Seed - Std Dev: {results_df['With Seed'].std():.6f}")
print("\n💡 Notice: With seed, the score is IDENTICAL across all trials!")

---

## Part 4: Code Organization

### 📁 Project Structure

A well-organized project structure is essential for reproducibility.

#### Recommended Structure:

```
my-research-project/
│
├── README.md                 # Project overview and instructions
├── requirements.txt          # Python dependencies
├── environment.yml           # Conda environment (if using conda)
├── .gitignore               # Files to exclude from version control
│
├── data/                    # Data files
│   ├── raw/                 # Original, immutable data
│   ├── processed/           # Cleaned and processed data
│   └── README.md            # Data documentation
│
├── notebooks/               # Jupyter notebooks
│   ├── 01_exploration.ipynb
│   ├── 02_analysis.ipynb
│   └── outputs/             # Notebook outputs
│
├── src/                     # Source code
│   ├── __init__.py
│   ├── data_processing.py
│   ├── models.py
│   └── utils.py
│
├── tests/                   # Unit tests
│   └── test_data_processing.py
│
├── results/                 # Analysis outputs
│   ├── figures/
│   └── tables/
│
└── docs/                    # Documentation
    └── methodology.md
```

### Coding Best Practices

#### 1. Use Relative Paths

In [ ]:
# ❌ BAD: Absolute paths (not portable)
bad_path = "C:/Users/John/Documents/project/data.csv"

# ✅ GOOD: Relative paths (portable)
import os

# Get the project root
project_root = os.path.dirname(os.path.abspath("."))
data_dir = os.path.join(project_root, "data", "raw")
good_path = os.path.join(data_dir, "data.csv")

print(f"✅ Portable path: {good_path}")
print("\n💡 This will work on any machine, regardless of username or OS!")

#### 2. Write Modular, Reusable Functions

In [ ]:
# ❌ BAD: Repeated code
print("❌ BAD APPROACH:")
print(
    """data1 = pd.read_csv('file1.csv')
data1 = data1.dropna()
data1['value'] = (data1['value'] - data1['value'].mean()) / data1['value'].std()

data2 = pd.read_csv('file2.csv')
data2 = data2.dropna()
data2['value'] = (data2['value'] - data2['value'].mean()) / data2['value'].std()
"""
)

print("\n" + "=" * 60)

# ✅ GOOD: Reusable function
print("✅ GOOD APPROACH:")


def load_and_preprocess(filepath, normalize_col=None):
    """
    Load CSV and perform standard preprocessing.

    Parameters:
    -----------
    filepath : str
        Path to CSV file
    normalize_col : str, optional
        Column to normalize (z-score)

    Returns:
    --------
    pd.DataFrame
        Processed data
    """
    # Load data
    data = pd.read_csv(filepath)

    # Remove missing values
    data = data.dropna()

    # Normalize if specified
    if normalize_col and normalize_col in data.columns:
        mean = data[normalize_col].mean()
        std = data[normalize_col].std()
        data[normalize_col] = (data[normalize_col] - mean) / std

    return data


print(
    """def load_and_preprocess(filepath, normalize_col=None):
    # Load, clean, and normalize data
    ...

data1 = load_and_preprocess('file1.csv', normalize_col='value')
data2 = load_and_preprocess('file2.csv', normalize_col='value')
"""
)

print("\n💡 Benefits: DRY (Don't Repeat Yourself), easier to maintain, fewer bugs!")

#### 3. Document Everything

In [ ]:
def calculate_effect_size(group1, group2, method="cohen_d"):
    """
    Calculate effect size between two groups.

    Parameters:
    -----------
    group1 : array-like
        Values from group 1
    group2 : array-like
        Values from group 2
    method : str, default='cohen_d'
        Effect size method. Options: 'cohen_d', 'hedges_g'

    Returns:
    --------
    float
        Effect size value

    Notes:
    ------
    Cohen's d interpretation:
    - Small: d = 0.2
    - Medium: d = 0.5
    - Large: d = 0.8

    References:
    -----------
    Cohen, J. (1988). Statistical power analysis for the behavioral sciences.

    Examples:
    ---------
    >>> group1 = [1, 2, 3, 4, 5]
    >>> group2 = [2, 3, 4, 5, 6]
    >>> calculate_effect_size(group1, group2)
    0.632...
    """
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    n1, n2 = len(group1), len(group2)

    # Pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))

    if method == "cohen_d":
        return (mean1 - mean2) / pooled_std
    elif method == "hedges_g":
        # Hedges' g includes a correction for small sample sizes
        j = 1 - (3 / (4 * (n1 + n2) - 9))
        return ((mean1 - mean2) / pooled_std) * j
    else:
        raise ValueError(f"Unknown method: {method}")


# Example usage
np.random.seed(42)
control = np.random.normal(100, 15, 50)
treatment = np.random.normal(110, 15, 50)

effect_size = calculate_effect_size(control, treatment)
print(f"Cohen's d: {effect_size:.3f}")
print(
    f"Interpretation: {'Large' if abs(effect_size) >= 0.8 else 'Medium' if abs(effect_size) >= 0.5 else 'Small'} effect"
)

print("\n✅ Well-documented function with:")
print("   • Clear parameter descriptions")
print("   • Return value documentation")
print("   • Interpretation guidelines")
print("   • References to original papers")
print("   • Usage examples")

---

## Part 5: Data Versioning and Management

### 📊 Why Version Data?

Code isn't the only thing that changes - data does too!

- 🔄 **Data updates**: New data collected over time
- 🛠️ **Processing changes**: Improved cleaning methods
- 🐛 **Error corrections**: Fixing mistakes in data
- 🎯 **Different subsets**: Various analysis focuses

### Data Versioning Strategies:

#### 1. **File Naming Convention**
```
data_v1.0_2024-01-15.csv
data_v1.1_2024-02-20.csv  (minor cleaning update)
data_v2.0_2024-03-10.csv  (major schema change)
```

#### 2. **Separate Raw and Processed**
```
data/
├── raw/              # NEVER modify these!
│   └── survey_responses.csv
└── processed/        # Generated from raw
    ├── survey_cleaned_v1.csv
    └── survey_cleaned_v2.csv
```

#### 3. **Use DVC (Data Version Control)**
Like Git, but for data:
```bash
dvc init
dvc add data/large_dataset.csv
git add data/large_dataset.csv.dvc
git commit -m "Add dataset v1"
```

In [ ]:
# Create a data versioning helper

import hashlib
import json
from datetime import datetime


def create_data_manifest(data, description, version, author):
    """
    Create a manifest file documenting dataset properties.

    This helps track what changed between versions.

    Parameters:
    -----------
    data : pd.DataFrame
        The dataset
    description : str
        What this version contains
    version : str
        Version number (e.g., '1.0', '2.1')
    author : str
        Who created this version

    Returns:
    --------
    dict
        Manifest information
    """
    # Calculate hash of data (for integrity checking)
    data_hash = hashlib.md5(pd.util.hash_pandas_object(data, index=True).values).hexdigest()

    manifest = {
        "version": version,
        "created_at": datetime.now().isoformat(),
        "author": author,
        "description": description,
        "data_hash": data_hash,
        "shape": {"rows": len(data), "columns": len(data.columns)},
        "columns": list(data.columns),
        "dtypes": data.dtypes.astype(str).to_dict(),
        "missing_values": data.isnull().sum().to_dict(),
        "memory_usage_mb": data.memory_usage(deep=True).sum() / 1024**2,
    }

    return manifest


# Example: Version our sample data
np.random.seed(42)
sample_data = pd.DataFrame(
    {
        "user_id": range(1, 101),
        "age": np.random.randint(18, 80, 100),
        "score": np.random.randn(100) * 10 + 50,
        "category": np.random.choice(["A", "B", "C"], 100),
    }
)

manifest = create_data_manifest(
    data=sample_data,
    description="Initial dataset from user survey",
    version="1.0",
    author="Research Team",
)

print("📋 DATA MANIFEST")
print("=" * 60)
print(json.dumps(manifest, indent=2))

# Save manifest
manifest_path = os.path.join(output_dir, "data_manifest_v1.0.json")
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

print(f"\n✅ Manifest saved to {manifest_path}")
print("\n💡 Store this with your data to track changes over time!")

---

## Part 6: Reproducibility Checklist

### ✅ Before Sharing Your Research

Use this checklist to ensure your work is reproducible:

In [ ]:
# Generate a comprehensive reproducibility checklist


def create_reproducibility_checklist():
    """
    Generate a comprehensive reproducibility checklist.
    """
    checklist = {
        "🌍 Environment": [
            "☐ Virtual environment created (venv/conda)",
            "☐ requirements.txt or environment.yml included",
            "☐ Python version documented",
            "☐ OS and platform documented (if relevant)",
        ],
        "🎲 Randomness": [
            "☐ Random seeds set for all libraries",
            "☐ Seed values documented in code and paper",
            "☐ Stochastic processes clearly noted",
        ],
        "💾 Data": [
            "☐ Data sources documented",
            "☐ Data collection date recorded",
            "☐ Raw data preserved (never modified)",
            "☐ Data processing pipeline documented",
            "☐ Data version tracked",
            "☐ Synthetic/example data provided (if original data is private)",
        ],
        "📝 Code": [
            "☐ Code uses relative paths (not absolute)",
            "☐ Functions are well-documented with docstrings",
            "☐ Code is modular and reusable",
            "☐ No hard-coded values (use constants/config files)",
            "☐ Code is version controlled (Git)",
        ],
        "📂 Organization": [
            "☐ Clear project structure",
            "☐ README.md with setup instructions",
            "☐ Separate directories for raw/processed data",
            "☐ Results/outputs clearly organized",
        ],
        "📖 Documentation": [
            "☐ Installation instructions included",
            "☐ Usage examples provided",
            "☐ Methodological choices explained",
            "☐ Known limitations documented",
            "☐ License file included",
        ],
        "🧪 Testing": [
            "☐ Code runs from scratch on clean environment",
            "☐ Results match reported values",
            "☐ Someone else can run your code",
        ],
        "🌐 Sharing": [
            "☐ Code available in public repository (GitHub, GitLab)",
            "☐ Data available (or access instructions provided)",
            "☐ Permanent DOI obtained (Zenodo, FigShare)",
            "☐ Repository archived",
        ],
    }

    return checklist


checklist = create_reproducibility_checklist()

print("✅ REPRODUCIBILITY CHECKLIST")
print("=" * 60)
for category, items in checklist.items():
    print(f"\n{category}")
    for item in items:
        print(f"  {item}")

# Save as markdown
checklist_md = "# Reproducibility Checklist\n\n"
for category, items in checklist.items():
    checklist_md += f"## {category}\n\n"
    for item in items:
        checklist_md += f"- {item}\n"
    checklist_md += "\n"

with open(os.path.join(output_dir, "reproducibility_checklist.md"), "w") as f:
    f.write(checklist_md)

print(f"\n✅ Checklist saved to {output_dir}/reproducibility_checklist.md")

---

## Practice Exercises

### Exercise 1: Environment Documentation

**Task:** Document your current environment and create a requirements.txt file.

1. Run the `document_environment()` function
2. Export your current packages: `pip freeze > requirements.txt`
3. Review the requirements.txt and remove unnecessary packages

### Exercise 2: Reproducibility Test

**Task:** Test if your code is reproducible.

1. Create a simple analysis with random elements
2. Run it twice WITHOUT setting seeds - compare results
3. Run it twice WITH seeds - verify results match

### Exercise 3: Project Structure

**Task:** Reorganize a messy project.

Given this structure:
```
project/
├── analysis_final_FINAL_v2.ipynb
├── data.csv
├── data_cleaned.csv
└── plot1.png
```

Reorganize it into a proper structure with raw/processed data, organized notebooks, and clear outputs.

### Exercise 4: Data Versioning

**Task:** Create manifests for multiple data versions.

1. Load a dataset
2. Create version 1.0 manifest
3. Modify the data (add rows, change values)
4. Create version 2.0 manifest
5. Compare the manifests to see what changed

In [ ]:
# Exercise 2 Solution: Reproducibility Test

print("🎯 EXERCISE 2: Reproducibility Test")
print("=" * 60)

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create synthetic dataset
X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42
)

print("\n❌ Test 1: WITHOUT SEED (Non-reproducible)")
print("-" * 60)
results_no_seed = []
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = RandomForestClassifier(n_estimators=10)
    model.fit(X_train, y_train)
    accuracy = accuracy_score(y_test, model.predict(X_test))
    results_no_seed.append(accuracy)
    print(f"Run {i+1}: Accuracy = {accuracy:.4f}")

print(f"\nStandard Deviation: {np.std(results_no_seed):.6f}")

print("\n✅ Test 2: WITH SEED (Reproducible)")
print("-" * 60)
results_with_seed = []
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    model.fit(X_train, y_train)
    accuracy = accuracy_score(y_test, model.predict(X_test))
    results_with_seed.append(accuracy)
    print(f"Run {i+1}: Accuracy = {accuracy:.4f}")

print(f"\nStandard Deviation: {np.std(results_with_seed):.6f}")

print("\n" + "=" * 60)
print("💡 Conclusion:")
print(f"   Without seed: Results vary (std = {np.std(results_no_seed):.6f})")
print(f"   With seed: Results identical (std = {np.std(results_with_seed):.6f})")

---

## Summary and Key Takeaways

### 🎯 What We Learned

1. **Reproducibility Types**
   - Computational: Same code + data + environment = same results
   - Empirical: Same methodology + new data = consistent findings
   - Statistical: Same data + different analysis = robust conclusions

2. **Environment Management**
   - Use virtual environments (venv, conda, Docker)
   - Document all dependencies with version numbers
   - Record system information (Python version, OS, etc.)

3. **Random Seed Control**
   - Set seeds for ALL libraries at the start
   - Document seed values clearly
   - Use same seed across related experiments

4. **Code Organization**
   - Use clear project structure
   - Write modular, reusable functions
   - Use relative paths, never absolute
   - Document everything with docstrings

5. **Data Versioning**
   - Never modify raw data
   - Use version numbers and dates
   - Create data manifests
   - Consider DVC for large datasets

### 📚 Best Practices Summary

| Practice | Why It Matters | How To Do It |
|----------|---------------|-------------|
| **Set random seeds** | Ensures consistent results | `np.random.seed(42)` at start |
| **Use virtual environments** | Prevents dependency conflicts | `python -m venv myenv` |
| **Document dependencies** | Others can recreate environment | `pip freeze > requirements.txt` |
| **Relative paths** | Code works on any machine | `os.path.join()` instead of hard-coded paths |
| **Version data** | Track changes over time | Use naming conventions + manifests |
| **Test reproducibility** | Verify it works from scratch | Run in fresh environment |

### 🚀 Next Steps

1. **Apply the checklist** to your current project
2. **Create a template** project structure you can reuse
3. **Practice** setting up reproducible environments
4. **Share your code** on GitHub with complete documentation

### 💡 Remember

> "If you can't reproduce it, it's not science - it's anecdote."

Reproducibility is not just good practice - it's essential for trustworthy research!

---

## Additional Resources

### Tools
- **DVC**: Data Version Control - https://dvc.org/
- **Docker**: Containerization - https://www.docker.com/
- **Conda**: Package management - https://docs.conda.io/
- **Poetry**: Dependency management - https://python-poetry.org/

### Reading
- "The Practice of Reproducible Research" (free online book)
- "Good Enough Practices in Scientific Computing" (Wilson et al., 2017)
- "Ten Simple Rules for Reproducible Computational Research" (Sandve et al., 2013)

### Guides
- The Turing Way: Guide for Reproducible Research
- ROpenSci Reproducibility Guide
- Project TIER Protocol

---

**Next Module:** [08_documentation_version_control.ipynb](08_documentation_version_control.ipynb) - Learn about documenting your research and using version control effectively!